In [1]:
%matplotlib widget

import time
import os
import sys
import warnings

from matplotlib import pyplot as plt
from ipywidgets import interact, interactive
from ipywidgets import FileUpload, Text, FloatRangeSlider, FloatSlider, Tab, HBox, VBox, Layout
from flowcorrection import Analysis
from shot import Shot

ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                   exception_only=False, running_compiled_code=False):
    etype, value, tb = sys.exc_info()
    def ignore(x, y):
        return []
    return ipython._showtraceback(etype, value, ignore(etype, value))

ipython.showtraceback = hide_traceback


def make_interactive(a: Analysis):
    def update_flow(correction_val):
        a._update_flow(a.shot.flow, correction_val)
    def update_window(window_value):
        a._update_window(window_value)

    # range slider (optimization window)
    end_time = a.shot.elapsed[-1]
    opt_window_slider = FloatRangeSlider(value=a._initial_window, min=0, max=end_time, step=0.1, readout=False, description='opt. window', layout=Layout(width='90%'))
    interact(update_window, window_value=opt_window_slider)
    # vertical slider (correction value)
    correction_slider = FloatSlider(value=1.0, min=0.3, max=2.5, step=0.01, readout=False, description='correction value', layout=Layout(width='90%'))
    correction_val = Text(value='', description='', layout=Layout(width='15%'))
    def set_correction_text(value):
        if a.shot.current_calibration == 1.0:
            value = 'x%.02f' % correction_slider.value
            correction_val.value = value
        else:
            value = 'x%.03f\n(%.02f)' % (a.shot.current_calibration * correction_slider.value, correction_slider.value)
            correction_val.value = value
    correction_slider.observe(set_correction_text, names='value')
    correction_box = VBox([interactive(update_flow, correction_val=correction_slider), correction_val], layout=Layout(width='100%'))
    display(correction_box)
    # TODO: +/- buttons (correction value)


def from_url(visualizer_url: str):
    if not visualizer_url.startswith('http://') and not visualizer_url.startswith('https://'):
        raise RuntimeError('invalid shot url!')
    s = Shot.parse_visualizer(visualizer_url)
    a = Analysis(s)
    a.show(False)
    make_interactive(a)

def from_file(uploaded):
    tempfile_name = 'uploaded_%d.shot' % time.time_ns()
    try:
        with open(tempfile_name, 'wb') as temp_shot_file:
            content = next(iter(uploaded.values()))['content']
            temp_shot_file.write(content)
            temp_shot_file.flush()
        with open(tempfile_name, 'r', encoding='utf-8') as shot_file:
            s = Shot.parse(shot_file)
            a = Analysis(s)
            a.show(False)
            make_interactive(a)
    finally:
        tempfile_abs_path = os.path.abspath(tempfile_name)
        if os.path.isfile(tempfile_abs_path):
            os.remove(tempfile_abs_path)

warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = (14,8)

tab = Tab()
for idx, s in enumerate(['from .shot file', 'from visualizer.coffee']):
    tab.set_title(idx, s)
tab_children = [
    interactive(from_file,
                uploaded=FileUpload(accept='.shot', multiple=False)),
    interactive(from_url, {'manual': True, 'manual_name': 'Run'},
                visualizer_url=Text('https://visualizer.coffee/shots/...').value)
]
tab.children = [VBox(children=(tab_children[0],)), VBox(children=(tab_children[1],))]
display(tab)